### Created on 2023
### @author: S.W

PyTorch는 파이썬 기반의 오픈소스 머신러닝 프레임워크로, 텐서 연산 및 그래프 생성, 자동 미분 등을 지원하여 딥러닝 모델의 학습 및 추론을 쉽게 구현할 수 있습니다. 이번에는 PyTorch 공식 홈페이지에서 제공하는 튜토리얼 중 하나인 "예제로 배우는 PyTorch"를 기반으로 PyTorch의 기초적인 개념과 사용 방법을 설명하겠습니다.

## 데이터셋 다운로드 및 전처리

첫 번째로, PyTorch에서 제공하는 torchvision 패키지를 이용해 데이터셋을 다운로드하고 전처리합니다. 이 예제에서는 MNIST 숫자 이미지 데이터셋을 사용합니다. torchvision.transforms.Compose를 사용해 다양한 전처리 기능을 조합할 수 있습니다. 이 예제에서는 이미지를 tensor로 변환하고, normalization을 수행합니다.

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

# 데이터셋 다운로드 및 전처리
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

ModuleNotFoundError: No module named 'torch'

위 코드에서는 torchvision.transforms.Compose를 사용하여 데이터 전처리 기능을 조합합니다. 먼저 ToTensor() 함수를 사용하여 이미지를 tensor로 변환하고, Normalize() 함수를 사용하여 이미지의 pixel 값을 -1~1 범위로 정규화합니다. MNIST 데이터셋을 다운로드하고, train=True인 경우 trainset으로, train=False인 경우 testset으로 데이터를 로드합니다. 그리고 DataLoader를 사용하여 데이터를 배치 단위로 로드합니다.

## 신경망 모델 정의

두 번째로, PyTorch에서는 신경망 모델을 정의하기 위해 nn.Module 클래스를 상속받아 모델을 정의합니다. 이 예제에서는 Convolutional Neural Network (CNN)을 사용합니다. nn.Conv2d, nn.MaxPool2d 등의 함수를 사용하여 CNN 레이어를 생성합니다. forward 함수를 정의하여 입력을 모델에 전달하면 출력을 반환하도록 합니다.



In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

net = Net()
print(net)

## 데이터 로드 및 학습

세 번째로, 모델을 학습시키기 위해 데이터셋을 준비하고 모델을 학습시킵니다. 이 예제에서는 MNIST 데이터셋을 사용합니다.

In [ ]:
import torch.optim as optim

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Train the network
for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs = inputs.reshape(-1, 28*28)

        # forward
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        
        # backward
        optimizer.zero_grad() # zero the parameter gradients
        loss.backward()
        optimizer.step() # optimize

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

위 코드에서 nn.CrossEntropyLoss()를 사용하여 분류 문제에서 사용하는 교차 엔트로피 손실 함수를 정의합니다. optim.SGD를 사용하여 Stochastic Gradient Descent (SGD) 최적화 알고리즘을 사용합니다. lr은 학습률을, momentum은 SGD에 사용하는 모멘텀 값을 설정합니다.

학습 과정에서는 trainloader에서 배치를 하나씩 가져옵니다. inputs와 labels를 가져온 다음, optimizer.zero_grad()를 사용하여 가중치의 변화도를 초기화합니다. net에 inputs를 전달하여 outputs를 계산합니다. criterion을 사용하여 outputs와 labels의 손실을 계산합니다. loss.backward()를 사용하여 역전파를 수행합니다. 마지막으로 optimizer.step()을 호출하여 최적화를 수행합니다.

매 에폭마다 running loss를 계산하여 출력합니다. i % 2000 == 1999 조건문을 사용하여 매 2000 배치마다 running loss를 출력합니다.

마지막으로, 학습이 완료되면 Finished Training을 출력합니다.

## 테스트 데이터 예측 및 평가

In [ ]:
# Test the network on the test data
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.reshape(-1, 28*28)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %f %%' % (
    100 * correct / total))

테스트 데이터셋을 사용하여 학습된 모델의 성능을 평가합니다. torch.no_grad()를 사용하여 추론 단계에서의 변화도 계산을 방지합니다. testloader에서 배치를 하나씩 가져옵니다. net에 images를 전달하여 outputs를 계산합니다. torch.max()를 사용하여 outputs에서 각 샘플별로 최대값을 가지는 인덱스를 가져옵니다. 이를 통해 예측된 라벨 predicted를 구합니다.

예측된 라벨 predicted와 실제 라벨 labels를 비교하여 올바르게 예측된 샘플의 개수를 correct에 누적합니다. 모든 샘플의 개수는 total에 누적합니다. 마지막으로, 전체 샘플 중에서 올바르게 예측한 샘플의 비율인 정확도를 출력합니다.

## 학습한 모델 저장

In [ ]:
PATH = './mnist_net.pth'
torch.save(net.state_dict(), PATH)

torch.save() 함수를 사용하여 모델의 state_dict를 파일에 저장합니다. state_dict는 모델의 학습 가능한 매개변수들을 포함합니다.

이제 모델을 저장했으므로, 나중에 다시 불러와서 사용할 수 있습니다. 다음과 같이 저장된 모델을 불러와서 사용할 수 있습니다.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

# 데이터셋 다운로드 및 전처리
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

net = Net()
print(net)

In [ ]:
PATH = './mnist_net.pth'

net = Net()
net.load_state_dict(torch.load(PATH))

Net() 클래스로 빈 모델을 생성한 다음, load_state_dict() 메소드를 사용하여 저장된 state_dict를 불러와서 모델에 적용합니다. 이제 불러온 모델을 사용하여 예측을 수행할 수 있습니다.

In [ ]:
# Test the network on the test data
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.reshape(-1, 28*28)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %f %%' % (
    100 * correct / total))